We need to install the leafmap client for data visualization in the notebook.

**Note:** This is set to run silently so you will not see an output when executing this cell. If you'd like to ensure the package downloads successfully remove the `--quiet` flag

![HLS Training](../images/HLS-inference.png)

In [ ]:
# Cell 1: Install required packages
!pip install ipyleaflet numpy azureml-core azure-storage-blob --quiet

Import the python libraries required for running the script

In [ ]:
# Cell 2: Import required libraries
import json
import ipyleaflet
import numpy as np
import requests
from azureml.core import Workspace, Model, Webservice
from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential

In [ ]:
try:
    ws = Workspace.from_config()
    print(f"Found workspace: {ws.name}")
except:
    # Initialize manually if needed
    ws = Workspace(
        subscription_id='your-subscription-id',
        resource_group='your-resource-group',
        workspace_name='your-workspace-name'
    )

In [ ]:
# Cell 4: Configuration for Azure ML inference
# UPDATE THESE VALUES:
STORAGE_ACCOUNT_NAME = 'your-storage-account'
CONTAINER_NAME = 'hls-models'
MODEL_NAME = 'your-identifier-hls-foundation'  # Replace with your registered model name
ENDPOINT_NAME = 'hls-inference-endpoint'  # Your deployed endpoint name

# Alternative: Direct Azure ML endpoint URL (if using managed endpoints)
INFERENCE_URL = f'https://{ENDPOINT_NAME}.{ws.location}.inference.ml.azure.com/'

In [ ]:
# Cell 5: Event details dictionary (unchanged)
EVENT_DETAILS = {
    'mongolian_fire': {
        'center_lon': 119.3,
        'center_lat': 47.1,
        'default_zoom': 8,
        'start_date': '2022-04-19T00:00:00Z',
        'end_date': '2022-04-19T23:59:59Z'
    },
    'new_mexico_black_fire': {
        'center_lon': -107.5,
        'center_lat': 33.5,
        'default_zoom': 10,
        'start_date': '2022-05-16T00:00:00Z',
        'end_date': '2022-06-10T23:59:59Z'
    },
    'alberta_fire': {
        'center_lon': -124.2,
        'center_lat': 61.8,
        'default_zoom': 8,
        'start_date': '2023-05-27T00:00:00Z',
        'end_date': '2023-05-28T23:59:59Z'
    },
    'maui_fire': {
        'center_lon': -156.659394,
        'center_lat': 20.886984,
        'default_zoom': 12,
        'start_date': '2023-08-13T00:00:00Z',
        'end_date': '2023-08-13T23:59:59Z'
    }
}

In [ ]:
# Cell 6: Select event
event = 'maui_fire'
event_details = EVENT_DETAILS[event]
print(f"Selected event: {event}")

In [ ]:
# Cell 7: Setup tile layers (unchanged)
datestring = event_details['start_date']
HLSL30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_L30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'
HLSS30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_S30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'


In [ ]:
# Cell 8: Create interactive map
from ipyleaflet import Map, TileLayer, DrawControl, GeoJSON

layer = TileLayer(url=HLSL30_TILE_LAYER, attribution='NASA', name='HLSL30', opacity=1)
draw_control = DrawControl()

map = Map(
    default_tiles=layer,
    center=(event_details['center_lat'], event_details['center_lon']), 
    zoom=event_details['default_zoom']
)

draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d", 
        "fillOpacity": 0.3
    }
}

hlsl30_tile_layer = TileLayer(url=HLSL30_TILE_LAYER, name='HLSL30', attribution='NASA')
hlss30_tile_layer = TileLayer(url=HLSS30_TILE_LAYER, name='HLSS30', attribution='NASA')

map.add_layer(hlsl30_tile_layer)
map.add_layer(hlss30_tile_layer)
map.add(draw_control)

# Store drawn shapes
drawn_shapes = []

def handle_draw(self, action, geo_json):
    if action == 'created':
        drawn_shapes.append(geo_json)
        print("Shape added.")

draw_control.on_draw(handle_draw)
map

Map(center=[20.886984, -156.659394], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [ ]:
# Cell 9: Extract bounding box from drawn shape
def bbox_from_geojson(bbox):
    """Get coordinates of bounding box from GeoJSON"""
    coordinates = np.asarray(bbox['geometry']['coordinates'])
    lats = coordinates[:, :, 1]  
    lons = coordinates[:, :, 0]
    return [lons.min(), lats.min(), lons.max(), lats.max()]

# Get bounding box (run after drawing on map)
if drawn_shapes:
    bbox = bbox_from_geojson(drawn_shapes[0])
    print(f"Bounding box: {bbox}")
else:
    print("⚠️  Please draw a rectangle on the map first!")

In [ ]:
# Cell 10: Setup Azure ML model and config references
# Update these values based on your training notebook outputs:
identifier = "your-identifier"  # Same as used in training notebook
BLOB_CONTAINER = "hls-data"  # Your blob container name

# Azure blob URLs for model and config
config_blob_url = f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{BLOB_CONTAINER}/configs/{identifier}-burn_scars_Prithvi_100M.py"
model_blob_url = f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{BLOB_CONTAINER}/models/{identifier}-workshop.pth"


In [ ]:
# Cell 11: Function to get Azure ML endpoint authentication
def get_azure_ml_headers():
    """Get authentication headers for Azure ML endpoint"""
    try:
        # For managed identity (when running in Azure)
        from azure.identity import DefaultAzureCredential
        from azure.core.credentials import AccessToken
        
        credential = DefaultAzureCredential()
        token = credential.get_token("https://ml.azure.com/.default")
        
        return {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {token.token}'
        }
    except:
        # Fallback: Use service principal or key-based auth
        # You'll need to set up authentication keys in your environment
        return {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {ws.get_details()["workspaceId"]}'  # Simplified - replace with actual token
        }

In [ ]:
 Cell 12: Make inference request to Azure ML endpoint
if 'bbox' in locals() and bbox:
    # Prepare payload for Azure ML endpoint
    payload = {
        "data": {
            "config_path": config_blob_url,
            "model_path": model_blob_url, 
            "model_type": "burn_scars",
            "date": event_details['start_date'].split('T')[0],
            "bounding_box": bbox
        }
    }
    
    # Get authentication headers
    headers = get_azure_ml_headers()
    
    print(f"Making inference request to: {INFERENCE_URL}")
    print(f"Payload: {json.dumps(payload, indent=2)}")
    
    try:
        # Option 1: Call Azure ML managed endpoint
        response = requests.post(
            f"{INFERENCE_URL}score",  # Azure ML endpoint pattern
            headers=headers,
            json=payload,
            timeout=300  # 5 minute timeout for processing
        )
        
        if response.status_code == 200:
            predictions = response.json()
            print("✅ Inference successful!")
        else:
            print(f"❌ Error: {response.status_code} - {response.text}")
            
    except Exception as e:
        print(f"❌ Request failed: {str(e)}")
        
        # Option 2: Fallback to local model inference (if model is available locally)
        print("Trying local inference as fallback...")
        
        try:
            # Load model locally and run inference
            model = Model(ws, name=MODEL_NAME)
            model_path = model.download(target_dir='./temp_model', exist_ok=True)
            
            # Your local inference code here...
            predictions = {"predictions": {"type": "FeatureCollection", "features": []}}
            print("✅ Local inference completed")
            
        except Exception as local_error:
            print(f"❌ Local inference also failed: {str(local_error)}")
            predictions = None

else:
    print("⚠️  Please draw a bounding box on the map first!")


In [ ]:
# Cell 13: Display results on map
if 'predictions' in locals() and predictions:
    try:
        geojson = predictions.get('predictions', predictions)
        
        detection_map = Map(
            center=(event_details['center_lat'], event_details['center_lon']), 
            zoom=event_details['default_zoom']
        )
        
        detection_map.add_layer(hlsl30_tile_layer)
        detection_map.add_layer(hlss30_tile_layer)
        
        # Add predictions as overlay
        if geojson:
            detection_map.add_layer(GeoJSON(data=geojson))
            print("✅ Results displayed on map")
        else:
            print("⚠️  No predictions to display")
            
        display(detection_map)
        
    except Exception as e:
        print(f"❌ Error displaying results: {str(e)}")

In [ ]:
# Cell 14: Alternative - Direct model inference using Azure ML SDK
"""
# Alternative approach using Azure ML SDK for batch inference
from azureml.core import Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

def run_local_inference():
    # Download registered model
    model = Model(ws, name=MODEL_NAME)
    model_path = model.download(target_dir='./downloaded_model', exist_ok=True)
    
    # Load your inference script and run prediction
    # This would use your actual model loading and inference code
    
    return {"predictions": "your_geojson_results"}

# Uncomment to use local inference
# predictions = run_local_inference()
"""

print("🎉 Inference notebook complete!")
print("\nKey differences from AWS version:")
print("✅ Uses Azure ML managed endpoints instead of custom API")  
print("✅ Authentication via Azure AD instead of API keys")
print("✅ Model and config loaded from Azure Blob Storage")
print("✅ Integrated with Azure ML model registry")